In [1]:
import numpy as np
import pandas as pd
from keras_tqdm import TQDMNotebookCallback

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import codecs
from tqdm import tqdm

from nltk.tokenize import RegexpTokenizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Conv1D, Conv2D, MaxPooling1D, GlobalMaxPool1D, Bidirectional, GlobalMaxPooling1D
from keras.layers import LSTM, GRU, Dropout , BatchNormalization, Embedding, Flatten, GlobalAveragePooling1D, concatenate, Input

Using TensorFlow backend.


In [9]:
train = pd.read_csv('train.csv')
train.dropna(inplace=True)
from sklearn.model_selection import train_test_split
train, test = train_test_split(train, random_state=42, test_size=0.30, shuffle=True)

list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
#list_classes =list_classes[0:100000]

y_train = train[list_classes]

In [10]:
y_test = test[list_classes]

In [11]:
y_train = train[list_classes]

In [12]:
len(train[list_classes])

111699

In [13]:
len(test[list_classes])

47872

In [14]:
train.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,111699.000000,111699.000000,111699.000000,111699.000000,111699.000000,111699.000000
mean,0.095901,0.009928,0.052758,0.003062,0.049132,0.008711
std,0.294456,0.099146,0.223551,0.055249,0.216145,0.092925
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [15]:
#test = pd.read_csv('test.csv')
#test.dropna(inplace=True)

In [16]:
test.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,47872.000000,47872.000000,47872.000000,47872.000000,47872.000000,47872.000000
mean,0.095714,0.010152,0.053392,0.002841,0.049904,0.009024
std,0.294201,0.100246,0.224817,0.053225,0.217749,0.094566
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [17]:
train.isnull().any()

id               False
comment_text     False
toxic            False
severe_toxic     False
obscene          False
threat           False
insult           False
identity_hate    False
dtype: bool

In [18]:
test.isnull().any()

id               False
comment_text     False
toxic            False
severe_toxic     False
obscene          False
threat           False
insult           False
identity_hate    False
dtype: bool

In [19]:
porter = PorterStemmer()
stop_words = set(stopwords.words('english'))
stop_words.update(['.', ',', '"', "'", ':', ';', '(', ')', '[', ']', '{', '}'])

In [20]:

MAX_NB_WORDS = 100000
max_seq_len = 50

raw_docs_train = train['comment_text'].tolist()
raw_docs_test = test['comment_text'].tolist()
num_classes = len(list_classes)
print raw_docs_train[0]
tokenizer = RegexpTokenizer(r'\w+')

print("pre-processing train data...")
processed_docs_train = []
for doc in tqdm(raw_docs_train):
    tokens = tokenizer.tokenize(doc)
    filtered = [word for word in tokens if word not in stop_words]
    processed_docs_train.append(" ".join(filtered))

print("pre-processing test data...")
processed_docs_test = []
for doc in tqdm(raw_docs_test):
    tokens = tokenizer.tokenize(doc)
    filtered = [word for word in tokens if word not in stop_words]
    processed_docs_test.append(" ".join(filtered))


print("tokenizing input data...")
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True, char_level=False)
tokenizer.fit_on_texts(processed_docs_train + processed_docs_test)  #leaky
word_seq_train = tokenizer.texts_to_sequences(processed_docs_train)
word_seq_test = tokenizer.texts_to_sequences(processed_docs_test)
word_index = tokenizer.word_index
print("dictionary size: ", len(word_index))

#pad sequences
word_seq_train = sequence.pad_sequences(word_seq_train, maxlen=max_seq_len)
word_seq_test = sequence.pad_sequences(word_seq_test, maxlen=max_seq_len)

print("Done !!")

  2%|▏         | 1856/111699 [00:00<00:05, 18556.66it/s]

"

Conflict of interest note
By your user name, it appears that you represent a company or organization. Please read our conflict of interest guidelines as well as our FAQ for businesses. We welcome your contributions here, but please refrain from writing about your own company's services and personnel. Thanks, and happy editing! ''''''Speaketh "
pre-processing train data...


  3%|▎         | 1541/47872 [00:00<00:03, 15409.87it/s]

pre-processing test data...


100%|██████████| 47872/47872 [00:02<00:00, 16661.97it/s]


tokenizing input data...
('dictionary size: ', 187702)
Done !!


In [21]:
len(word_seq_train)

111699

In [22]:
len(word_index)

187702

In [23]:
embed_dim = 300

#load embeddings
print('loading first word embeddings...')
embeddings_index = {}
f = codecs.open('EMB/glove.6B.300d.txt', encoding='utf-8')

for line in tqdm(f):
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('found %s word vectors' % len(embeddings_index))

0it [00:00, ?it/s]

loading first word embeddings...


400000it [02:04, 3217.01it/s]

found 400000 word vectors


In [24]:
print('preparing embedding matrix...')
words_not_found = []
nb_words = min(MAX_NB_WORDS, len(word_index))
embedding_matrix_glove = np.zeros((nb_words, embed_dim))
for word, i in word_index.items():
    if i >= nb_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if (embedding_vector is not None) and len(embedding_vector) > 0:
        # words not found in embedding index will be all-zeros.
        embedding_matrix_glove[i] = embedding_vector
    else:
        words_not_found.append(word)
print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix_glove, axis=1) == 0))

preparing embedding matrix...


/home/sinchan/.local/lib/python2.7/site-packages/ipykernel_launcher.py:8: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  


number of null word embeddings: 31473


In [20]:
from gensim.models import KeyedVectors
w2v = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin",binary = True,limit=50000)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/sinchan/.local/lib/python2.7/site-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/home/sinchan/.local/lib/python2.7/site-packages/IPython/core/ultratb.py", line 313, in wrapped
    return f(*args, **kwargs)
  File "/home/sinchan/.local/lib/python2.7/site-packages/IPython/core/ultratb.py", line 358, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))
  File "/usr/lib/python2.7/inspect.py", line 1051, in getinnerframes
    framelist.append((tb.tb_frame,) + getframeinfo(tb, context))
  File "/usr/lib/python2.7/inspect.py", line 1011, in getframeinfo
    filename = getsourcefile(frame) or getfile(frame)
  File "/usr/lib/python2.7/inspect.py", line 453, in getsourcefile
    if hasattr(getmodule(object, filename), '__loader__'):
  File "/usr/lib/python2.7/inspect.py", line 496, in getmodule
   

IndexError: string index out of range

In [11]:
nb_words = len(word_index)
EMBEDDING_DIM =300
embedding_matrix = np.zeros((nb_words+1, EMBEDDING_DIM))

i = 0
x_g = x_w = 0
for word in word_index:   
    try:
        embedding_matrix[word_index[word]][:300] = w2v[word]
    except KeyError:
        x_w += 1

NameError: name 'w2v' is not defined

In [17]:
len(embedding_matrix_glove)

100000

In [18]:
max_seq_len

50

In [35]:
inp = Input(shape=(max_seq_len, ))
emb_glove = Embedding(len(embedding_matrix_glove), 300, 
                weights=[embedding_matrix_glove], input_length = max_seq_len, trainable=False)(inp)

In [36]:
x = Bidirectional(LSTM(25,return_sequences=True))(emb_glove)
x = Bidirectional(GRU (25, return_sequences=True))(x)
avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
out = concatenate([avg_pool, max_pool])

out = Dense(10, activation="relu")(out)
out = Dense(y_train.shape[1], activation="sigmoid")(out)

model2 = Model(inputs=inp, outputs=out)

model2.compile(loss='binary_crossentropy', optimizer = keras.optimizers.Adam(lr=0.001),
              metrics=['accuracy'])

In [21]:
Y_test = pd.read_csv('test_labels.csv')
y_test = Y_test[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values
y_test=y_test[0:100000]

In [23]:
model.fit(word_seq_train, y_train, validation_data=(np.array(word_seq_test),np.array(y_test)),epochs=50, batch_size=80, shuffle=True,verbose = 2)

Train on 100000 samples, validate on 100000 samples
Epoch 1/50
 - 405s - loss: 0.0721 - acc: 0.9769 - val_loss: -2.7931e+00 - val_acc: 0.4065
Epoch 2/50
 - 401s - loss: 0.0507 - acc: 0.9815 - val_loss: -3.1115e+00 - val_acc: 0.4033
Epoch 3/50
 - 401s - loss: 0.0472 - acc: 0.9825 - val_loss: -3.3943e+00 - val_acc: 0.4055
Epoch 4/50
 - 404s - loss: 0.0442 - acc: 0.9834 - val_loss: -3.2545e+00 - val_acc: 0.4050
Epoch 5/50
 - 401s - loss: 0.0413 - acc: 0.9843 - val_loss: -3.5338e+00 - val_acc: 0.4056
Epoch 6/50
 - 401s - loss: 0.0386 - acc: 0.9853 - val_loss: -3.9460e+00 - val_acc: 0.4070
Epoch 7/50
 - 401s - loss: 0.0359 - acc: 0.9862 - val_loss: -3.9251e+00 - val_acc: 0.4062
Epoch 8/50
 - 402s - loss: 0.0335 - acc: 0.9871 - val_loss: -4.0131e+00 - val_acc: 0.4053
Epoch 9/50
 - 401s - loss: 0.0311 - acc: 0.9879 - val_loss: -4.2224e+00 - val_acc: 0.4051
Epoch 10/50
 - 401s - loss: 0.0291 - acc: 0.9888 - val_loss: -4.1975e+00 - val_acc: 0.4044
Epoch 11/50
 - 401s - loss: 0.0271 - acc: 0.989

In [ ]:
model2.fit(word_seq_train, y_train, validation_data=(np.array(word_seq_test),np.array(y_test)),epochs=10, batch_size=80, shuffle=True,verbose = 2)

Train on 111699 samples, validate on 47872 samples
Epoch 1/10
 - 675s - loss: 0.0744 - acc: 0.9767 - val_loss: 0.0536 - val_acc: 0.9809
Epoch 2/10
 - 786s - loss: 0.0504 - acc: 0.9817 - val_loss: 0.0501 - val_acc: 0.9819
Epoch 3/10
 - 927s - loss: 0.0466 - acc: 0.9827 - val_loss: 0.0487 - val_acc: 0.9823
Epoch 4/10


In [24]:
from keras.models import load_model
model.save('my_model.h5')

In [25]:
from keras import backend as K
from keras.engine.topology import Layer
from keras import backend as K
import os

def set_keras_backend(backend):

    if K.backend() != backend:
        os.environ['KERAS_BACKEND'] = backend
        reload(K)
        assert K.backend() == backend

set_keras_backend("theano")

Using Theano backend.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [26]:
from keras import initializers
import numpy as np
class AttLayer(Layer):
    def __init__(self, **kwargs):
        self.init = initializers.get('normal')
        super(AttLayer, self).__init__(** kwargs)

    def build(self, input_shape):
        assert len(input_shape)==3
        self.W = self.add_weight(name='kernel', 
                                  shape=(input_shape[-1],),
                                  initializer='normal',
                                  trainable=True)
        super(AttLayer, self).build(input_shape) 
     


    def call(self, x, mask=None):
        eij = K.tanh(K.dot(x, self.W))

        ai = K.exp(eij)
        weights = ai/K.sum(ai, axis=1).dimshuffle(0,'x')

        weighted_input = x*weights.dimshuffle(0,1,'x')
        return weighted_input.sum(axis=1)

    def compute_output_shape(self, input_shape):
        print input_shape[0]
        return (input_shape[0], input_shape[-1])

In [27]:
inp = Input(shape=(max_seq_len, ))
print "hello"
emb_glove = Embedding(len(embedding_matrix_glove), 300, 
                weights=[embedding_matrix_glove], input_length = max_seq_len, trainable=False)(inp)
print "hello"
x = Bidirectional(LSTM(25,return_sequences=True))(emb_glove)
l_gru = Bidirectional(GRU(25,return_sequences=True))(x)
print "hello"
l_att = AttLayer()(l_gru)
out = Dense(10, activation="relu")(l_att)
print "hello"
preds = Dense(6, activation='softmax')(out)


hello
hello
hello
None
hello


In [28]:
model = Model(inp, preds)

In [29]:
model.compile(loss='binary_crossentropy', optimizer = keras.optimizers.Adam(lr=0.001),
              metrics=['accuracy'])

In [30]:
model.fit(word_seq_train, y_train, validation_data=(np.array(word_seq_test),np.array(y_test)),epochs=10, batch_size=80, shuffle=True,verbose = 2)

Train on 111699 samples, validate on 47872 samples
Epoch 1/10
 - 769s - loss: 0.2231 - acc: 0.9633 - val_loss: 0.2213 - val_acc: 0.9626
Epoch 2/10
 - 680s - loss: 0.2197 - acc: 0.9641 - val_loss: 0.2201 - val_acc: 0.9644
Epoch 3/10
 - 683s - loss: 0.2181 - acc: 0.9646 - val_loss: 0.2196 - val_acc: 0.9630
Epoch 4/10
 - 684s - loss: 0.2170 - acc: 0.9653 - val_loss: 0.2196 - val_acc: 0.9630
Epoch 5/10
 - 674s - loss: 0.2160 - acc: 0.9656 - val_loss: 0.2203 - val_acc: 0.9625
Epoch 6/10
 - 676s - loss: 0.2150 - acc: 0.9661 - val_loss: 0.2203 - val_acc: 0.9633
Epoch 7/10
 - 678s - loss: 0.2140 - acc: 0.9665 - val_loss: 0.2214 - val_acc: 0.9636
Epoch 8/10
 - 675s - loss: 0.2131 - acc: 0.9670 - val_loss: 0.2228 - val_acc: 0.9646
Epoch 9/10
 - 677s - loss: 0.2122 - acc: 0.9673 - val_loss: 0.2224 - val_acc: 0.9633
Epoch 10/10
 - 677s - loss: 0.2112 - acc: 0.9678 - val_loss: 0.2232 - val_acc: 0.9639


In [ ]:
model.save('my_model2.h5')

In [ ]:
model2.save('my_model3.h5')

In [34]:
import pandas
tokenizer = RegexpTokenizer(r'\w+')
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True, char_level=False)
tokenizer.fit_on_texts(processed_docs_train + processed_docs_test) 
s="what the fuck"
arr=[]
arr.append(s)
s = tokenizer.texts_to_sequences(arr)
s = sequence.pad_sequences(s, maxlen=max_seq_len)
res=model.predict(s)
final=[]
for i in range(len(res)):
    for j in range(len(res[i])):
        final.append(str(int(res[i][j]*100))+"%")
l=[["toxic","severe_toxic","obscene","threat","insult","identity_hate"]]
pandas.DataFrame(final, l)

,0
toxic,47%
severe_toxic,0%
obscene,37%
threat,0%
insult,13%
identity_hate,0%
